In [1]:
#import libraries

# standard library
import os
import sys
from datetime import datetime
import time
import re

# csv, xls, pandas & json
import pandas as pd
import json
import csv
import xlrd

#twitter libraries
import tweepy
from tweepy import StreamListener
from tweepy import Stream
import preprocessor as p
from twitter_login_fn import oauth_login
from twitter_login_fn import appauth_login
from db_fn import save_to_DB
from db_fn import load_from_DB
# from tweet-preprocessor import clean,tokenize,parse

#MongoDB libraries
import pymongo
from pymongo import MongoClient

# #visualization
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
# import seaborn as sns
# %matplotlib inline

# import timeit
# import random

os.getcwd()

'/Users/sathishrajendiran/ist652-python'

In [15]:
# Connection to Mongo DB
try:
    client = MongoClient('localhost', 27017)
#             print ('Connected successfully!!!')
    print ('Authentication OK - You''re now connected to the MongoDB.\n')

    # use database named usgs or create it if not there already
    db = client.covid
    # create collection named earthquakes or create it if not there already
    coll = db.covidtweets

#   print('MongoDB: Tweets db with tweets collection has been created')
    print('MongoDB database: ' + str(db))
    print('MongoDB collection:' + str(coll))

except pymongo.errors.ConnectionFailure as e:
    print ('Could not connect to MongoDB: %s' % e )

Authentication OK - Youre now connected to the MongoDB.

MongoDB database: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'covid')
MongoDB collection:Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'covid'), 'covidtweets')


In [16]:
''' 
This main topic search function for Twitter using the python tweepy package
      Tries to get up to 100 results from the Twitter REST/Search API search function
        using the tweepy Cursor to repeat the twitter search api requests
      The query string may be a keyword or hashtag, or a set of them connected by or
        example:  query = "#CuseLAX OR CNYlacrosse"
        some queries require quotes on the command line
    Returns a list of json formatted tweets
'''


sys.argv=['twitter_simple_search.py',100,'lax','syracuselax']

arg1 = '#covid'
arg2 = 200
arg3 = 'covid'
arg4 = 'covidtweets'


'''
    Uses the tweepy Cursor to wrap a twitter api search for the query string
    Returns json formatted results
'''

def simple_search(api, query, max_results=20):
  # the first search initializes a cursor, stored in the metadata results,
  #   that allows next searches to return additional tweets
    search_results = [status for status in tweepy.Cursor(api.search, q=query).items(max_results)]

    # for each tweet, get the json representation
    tweets = [tweet._json for tweet in search_results]

    return tweets

# use a main so can get command line arguments
if __name__ == '__main__':
    # Make a list of command line arguments, omitting the [0] element
    # which is the script itself.
    args = sys.argv[1:]
 #   if not args or len(args) < 4:
 #       print('usage: python twitter_simple_search.py <query> <num tweets> <DB name> <collection name>')
 #       sys.exit(1)
    query = arg1
    num_tweets = int(arg2)
    DBname = arg3
    DBcollection = arg4

    # api = oauth_login()
    ''' if needed switch to using the appauth login to avoid rate limiting '''
    
    #create dummy dictionary
    di = {}
    # define file name
    infile = 'tw_credentials.xls'

    # Working with file
    try: 
        df = pd.read_excel(infile, encoding='utf-16')
        di = df.to_dict()
        print("Authentication data has been processed \n")
    except:
        print("Is the file in correct directory?")
    #assign it to variables
    for key,val in di.items():
        consumer_key = val[0]
        consumer_secret = val[1]
        access_token = val[2]
        access_secret = val[3]
    # test authentication
    try:
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        api.verify_credentials()
        print('Authentication OK - You''re now connected to the Twitter API.\n')
    except:
        print('Error during authentication')
    
#     api = appauth_login()
#     print ("Twitter Authorization: ", api)
    
    # access Twitter search
    result_tweets = simple_search(api, query, max_results=num_tweets)
    print ('Number of result tweets: ', len(result_tweets))

    # save the results in a database collection
    #   change names to lowercase because they are not case sensitive
    #   and remove special characters like hashtags and spaces (other special characters may also be forbidden)
    DBname = DBname.lower()
    DBname = DBname.replace('#', '')
    DBname = DBname.replace(' ', '')
    DBcollection = DBcollection.lower()
    DBcollection = DBcollection.replace('#', '')
    DBcollection = DBcollection.replace(' ', '')
    
    # use the save and load functions in this program
    save_to_DB(DBname, DBcollection, result_tweets)
    load_from_DB(DBname, DBcollection)
    # Done!
  

Authentication data has been processed 

Authentication OK - Youre now connected to the Twitter API.

Number of result tweets:  200
Saved 200 documents to DB collection covid covidtweets


In [19]:
# def print_tweet_data(tweets):
#     for tweet in tweets:
#         if tweet['retweet_count']<10:
#            print('\nDate: ',tweet['created_at'])
#            print('From: ',tweet['user']['name'])
#            print('Message: ',tweet['text'])
#            print('Retweet Count:',tweet['retweet_count'])

In [28]:
#search the first item from the collection
coll.find_one()

{'_id': ObjectId('5f3e16b75469bbcc8bad219a'),
 'created_at': 'Thu Aug 20 06:22:27 +0000 2020',
 'id': 1296331746741493761,
 'id_str': '1296331746741493761',
 'text': 'RT @MdcMasvingo: Masvingo General hosiptal got no testing kits about 16 student nurses tested Positive for #Covid and some looters are Sayi…',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'Covid', 'indices': [107, 113]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'MdcMasvingo',
    'name': 'MDC Alliance Masvingo',
    'id': 1138385964529856513,
    'id_str': '1138385964529856513',
    'indices': [3, 15]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1054333598512435200,
  'id_str': '105

In [34]:
# docs = coll.find()
# print('docs cursor ready!')

# print_tweet_data(docs)

docs cursor ready!

Date:  Thu Aug 20 06:22:27 +0000 2020
From:  Henry Ngara 🇿🇼
Message:  RT @MdcMasvingo: Masvingo General hosiptal got no testing kits about 16 student nurses tested Positive for #Covid and some looters are Sayi…
Retweet Count: 1

Date:  Thu Aug 20 06:22:20 +0000 2020
From:  Jorge De Los Ríos
Message:  Estamos actualmente viviendo en medio de una #pandemia. En todos lados hablamos del #COVID-19 pero existe otro viru… https://t.co/EA3opinkbn
Retweet Count: 0

Date:  Thu Aug 20 06:22:09 +0000 2020
From:  angiuoniluigi
Message:  RT @Adnkronos: #Covid, in #Germania mai così tanti nuovi casi in un giorno https://t.co/lNblPUYM5R https://t.co/VRdpUHODJn
Retweet Count: 1

Date:  Thu Aug 20 06:22:07 +0000 2020
From:  Aisha
Message:  The Kibo Code https://t.co/cTPErWXk8n

#smallbusiness #shopping #startup #dropshipping #socialmediamarketing… https://t.co/XIjTbS1tOK
Retweet Count: 0

Date:  Thu Aug 20 06:22:01 +0000 2020
From:  Mom's Choice Awards
Message:  Don't forget about ou

In [29]:
docs = coll.find()
print('docs cursor ready!')
tw_list =[]
doclist = [tweet for tweet in docs]
for tweet in doclist:
    id = tweet['id']
    user = tweet['user']['name']
    friends_count = tweet['user']['friends_count']
    followers = tweet['user']['followers_count']
    retweets = tweet['retweet_count']
    bg_color = tweet['user']['profile_background_color']
    datets = tweet['created_at']
    text = tweet['text']
    tw_list.append([id,user,friends_count,followers,retweets,bg_color,datets,text])
print('Number of docs: ',len(tw_list))

docs cursor ready!
Number of docs:  1600


In [30]:
#define column names
ColNames = ['id','user','friends_count','followers','retweets','bg_color','datets','text']
# Show all columns and do not truncate in the data frame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
tweetsDF = pd.DataFrame(tw_list,columns=ColNames)
print('Total Number of rows Processed: ',len(tweetsDF))

Total Number of rows Processed:  1600


In [31]:
##### datetime conversion
tweetsDF['datets'] = tweetsDF['datets'].astype('datetime64[ns]')
tweetsDF['date'] = tweetsDF['datets'].dt.date
tweetsDF['month'] = tweetsDF['datets'].dt.month
tweetsDF['monthday'] = tweetsDF['datets'].dt.day
tweetsDF['weekday'] = tweetsDF['datets'].dt.weekday
tweetsDF['dayname'] = tweetsDF['datets'].dt.day_name()
tweetsDF['monthname'] = tweetsDF['datets'].dt.month_name()
tweetsDF['hour'] = tweetsDF['datets'].dt.hour
tweetsDF['minute'] = tweetsDF['datets'].dt.minute
tweetsDF['secs'] = tweetsDF['datets'].dt.second
print('Date time conversion complete!')

Date time conversion complete!


In [32]:
#validate Number of retweets by values 
retweets = tweetsDF[tweetsDF['text'].str.startswith('RT')==True]
print('Number of retweets: ',len(retweets))
print('Percentage of retweets {}%'.format(round((len(retweets))/len(tweetsDF['text'])*100,2)))

retweets.head()

Number of retweets:  1202
Percentage of retweets 75.12%


,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
0,1296331746741493761,Henry Ngara 🇿🇼,729,1200,1,F5F8FA,2020-08-20 06:22:27,RT @MdcMasvingo: Masvingo General hosiptal got no testing kits about 16 student nurses tested Positive for #Covid and some looters are Sayi…,2020-08-20,8,20,3,Thursday,August,6,22,27
2,1296331672087154688,angiuoniluigi,7501,7120,1,000000,2020-08-20 06:22:09,"RT @Adnkronos: #Covid, in #Germania mai così tanti nuovi casi in un giorno https://t.co/lNblPUYM5R https://t.co/VRdpUHODJn",2020-08-20,8,20,3,Thursday,August,6,22,9
5,1296331637786136576,lizzie eldridge,2460,1819,6,ACDED6,2020-08-20 06:22:01,RT @TheShiftNews: ‘Jurassic’ film scales back production due to #COVID mishandling in #Malta https://t.co/yoV8IksbAq,2020-08-20,8,20,3,Thursday,August,6,22,1
6,1296331634698944512,sneha nair,190,39,2,F5F8FA,2020-08-20 06:22:01,RT @MumbaiMirror: #SevenHillsHospital has been fortified with 300 additional beds in view of #GaneshChaturthi. #COVID19 #COVID #Coronavirus…,2020-08-20,8,20,3,Thursday,August,6,22,1
7,1296331605531930624,Ivic ⭐⭐⭐,644,887,2,F5F8FA,2020-08-20 06:21:54,RT @adrianobusolin: Prima #Putin ha approvato il vaccino della Russia contro il #Covid. Poi Trump ha silurato Fauci. #Trump e Putin sono pr…,2020-08-20,8,20,3,Thursday,August,6,21,54


In [33]:
#validate Number of retweets by values 
act_tweets = tweetsDF[tweetsDF['text'].str.startswith('RT')==False]
print('Number of actual tweets: ',len(act_tweets))
print('Percentage of actualtweets {}%'.format(round((len(act_tweets))/len(tweetsDF['text'])*100,2)))

act_tweets.head()

Number of actual tweets:  398
Percentage of actualtweets 24.88%


,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
1,1296331717675032576,Jorge De Los Ríos,146,237,0,074542,2020-08-20 06:22:20,Estamos actualmente viviendo en medio de una #pandemia. En todos lados hablamos del #COVID-19 pero existe otro viru… https://t.co/EA3opinkbn,2020-08-20,8,20,3,Thursday,August,6,22,20
3,1296331663803166721,Aisha,336,488,0,F5F8FA,2020-08-20 06:22:07,The Kibo Code https://t.co/cTPErWXk8n\n\n#smallbusiness #shopping #startup #dropshipping #socialmediamarketing… https://t.co/XIjTbS1tOK,2020-08-20,8,20,3,Thursday,August,6,22,7
4,1296331638314545152,Mom's Choice Awards,1894,12655,0,313B91,2020-08-20 06:22:01,Don't forget about our #Free #Resources to Help #Parents &amp; #Teachers during #COVID from our amazing #honorees. New… https://t.co/A30VuV2TKS,2020-08-20,8,20,3,Thursday,August,6,22,1
10,1296331578642067458,Ahmedabad Mirror,284,35854,0,D9261C,2020-08-20 06:21:47,"City students want to be withdrawn from #Covid duty as ‘they are not qualified’, ask why other med students in Stat… https://t.co/IcYLHnhWxK",2020-08-20,8,20,3,Thursday,August,6,21,47
14,1296331550150123520,prabhu razdan,2195,1174,0,C0DEED,2020-08-20 06:21:40,"#Covid update #India Spike of 69,652 cases &amp; 977 fatalities in last 24 hours. Tally +ve cases 28,36,926 including 6… https://t.co/fKUCFfTmnK",2020-08-20,8,20,3,Thursday,August,6,21,40


In [23]:
#number of tweets by date
by_date = act_tweets.groupby(['date']).size().reset_index(name='counts')
by_date = by_date.set_index('date')
by_date.reset_index(level=0,inplace = True,drop=False)
by_date.index += 1
print('Tweets by calender date: ')
by_date

Tweets by calender date: 


,date,counts
1,2020-08-20,398


In [24]:
#number of tweets by hour of day
by_day_hour = act_tweets.groupby(['dayname','hour']).size().reset_index(name='counts')
by_day_hour = by_day_hour.set_index('dayname')
by_day_hour.reset_index(level=0,inplace = True,drop=False)
by_day_hour.index += 1
print('Tweets by hour of the day: ')
by_day_hour

Tweets by hour of the day: 


,dayname,hour,counts
1,Thursday,6,277
2,Thursday,22,39
3,Thursday,23,82


In [25]:
#count Number of tweets with retweets less than 10 
below10_RT = tweetsDF[tweetsDF['retweets']<10]
print('Number of retweets: ',len(below10_RT))
print('Percentage of retweets {}%'.format(round((len(below10_RT))/len(tweetsDF['text'])*100,2)))

below10_RT.head()

Number of retweets:  776
Percentage of retweets 48.5%


,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
0,1296331746741493761,Henry Ngara 🇿🇼,729,1200,1,F5F8FA,2020-08-20 06:22:27,RT @MdcMasvingo: Masvingo General hosiptal got no testing kits about 16 student nurses tested Positive for #Covid and some looters are Sayi…,2020-08-20,8,20,3,Thursday,August,6,22,27
1,1296331717675032576,Jorge De Los Ríos,146,237,0,074542,2020-08-20 06:22:20,Estamos actualmente viviendo en medio de una #pandemia. En todos lados hablamos del #COVID-19 pero existe otro viru… https://t.co/EA3opinkbn,2020-08-20,8,20,3,Thursday,August,6,22,20
2,1296331672087154688,angiuoniluigi,7501,7120,1,000000,2020-08-20 06:22:09,"RT @Adnkronos: #Covid, in #Germania mai così tanti nuovi casi in un giorno https://t.co/lNblPUYM5R https://t.co/VRdpUHODJn",2020-08-20,8,20,3,Thursday,August,6,22,9
3,1296331663803166721,Aisha,336,488,0,F5F8FA,2020-08-20 06:22:07,The Kibo Code https://t.co/cTPErWXk8n\n\n#smallbusiness #shopping #startup #dropshipping #socialmediamarketing… https://t.co/XIjTbS1tOK,2020-08-20,8,20,3,Thursday,August,6,22,7
4,1296331638314545152,Mom's Choice Awards,1894,12655,0,313B91,2020-08-20 06:22:01,Don't forget about our #Free #Resources to Help #Parents &amp; #Teachers during #COVID from our amazing #honorees. New… https://t.co/A30VuV2TKS,2020-08-20,8,20,3,Thursday,August,6,22,1


In [26]:
#count Number of tweets with retweets more than 100 
above100_RT = tweetsDF[tweetsDF['retweets']>100]
print('Number of retweets: ',len(above100_RT))
print('Percentage of retweets {}%'.format(round((len(above100_RT))/len(tweetsDF['text'])*100,2)))

above100_RT.head()

Number of retweets:  467
Percentage of retweets 29.19%


,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
12,1296331557918199808,Karen,343,115,199,C0DEED,2020-08-20 06:21:42,"RT @biobiobiobior: 🔴 France, 19 Août: “Il faut se calmer , ne pas s’affoler, il s’agit d’une maladie virale comme une autre, il faut se cal…",2020-08-20,8,20,3,Thursday,August,6,21,42
18,1296331461818118144,中津川亮,1877,2231,193,C0DEED,2020-08-20 06:21:19,RT @CllrBSilvester: Spanish media desperately trying to whip up fear of #COVID.\nDoctor is having none of it.\n'There is no emergency. \nHospi…,2020-08-20,8,20,3,Thursday,August,6,21,19
19,1296331460001951744,Rahul Choudhary,37,18,407,F5F8FA,2020-08-20 06:21:19,"RT @bhupendrachaube: ""SC &amp; Ministers are doing virtual meetings, but we are being forced to give exams amidst #Covid"" - JEE aspirant Jyotir…",2020-08-20,8,20,3,Thursday,August,6,21,19
21,1296331418998636545,"¡ma,pos ora!",4855,3010,748,F5F8FA,2020-08-20 06:21:09,RT @AnonymousMex_: Es más importante el escándalo del vídeo pedorro de #Lozoya patrocinado por @lopezobrador_ que los muertos por #COVID y…,2020-08-20,8,20,3,Thursday,August,6,21,9
34,1296331256075104256,kocher christian,2255,811,199,F5F8FA,2020-08-20 06:20:30,"RT @biobiobiobior: 🔴 France, 19 Août: “Il faut se calmer , ne pas s’affoler, il s’agit d’une maladie virale comme une autre, il faut se cal…",2020-08-20,8,20,3,Thursday,August,6,20,30


In [27]:
#count Number of tweets with retweets more than 100 
covid_tweets = tweetsDF[tweetsDF['text'].str.contains('covid')]
print('Number of retweets: ',len(covid_tweets))
print('Percentage of retweets {}%'.format(round((len(covid_tweets))/len(tweetsDF['text'])*100,2)))

covid_tweets.head()

Number of retweets:  192
Percentage of retweets 12.0%


,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
33,1296331304414269440,दिनेश चंद्र गर्ग,328,124,1,F5F8FA,2020-08-20 06:20:42,RT @NewsOnCards: #tamilnaducases #covid #COVID__19 #COVID19India #Covid_19 #COVID19vaccine #newsoncards #IndiaFightsCorona #India #Indian #…,2020-08-20,8,20,3,Thursday,August,6,20,42
40,1296331176836239362,Jonathan Davis FPFS FCII 📉📈 #KBF,231,3514,3,C0DEED,2020-08-20 06:20:11,RT @UraniumSir: Funny cause it’s true\n\n#covid\n#coronavirus https://t.co/46cji3Me5J,2020-08-20,8,20,3,Thursday,August,6,20,11
43,1296331117742751744,Essie,985,1432,2,F5F8FA,2020-08-20 06:19:57,RT @PublicPathways: Adolescents and Youth deserve special attention during #covid pandemic. Today we ask govt's to implement #RHcovid guide…,2020-08-20,8,20,3,Thursday,August,6,19,57
59,1296330846291595265,SemperAdamantes 🇮🇹,1539,1679,135,F5F8FA,2020-08-20 06:18:53,RT @fdragoni: Oggi in tutta la Toscana abbiamo 25 ricoverati col #covid (di cui 5 in terapia intensiva). Sostenere che vi sia un allarme #c…,2020-08-20,8,20,3,Thursday,August,6,18,53
61,1296330781451849728,France 3 Pays de la Loire,920,24506,0,131516,2020-08-20 06:18:37,#Coronavirus rebond du #covid en #Sarthe\n►https://t.co/kZ54swWbCm https://t.co/uOy9B0sNSP,2020-08-20,8,20,3,Thursday,August,6,18,37
